Paso 3 de la Práctica

In [0]:
from pyspark.sql import SparkSession

# Crear la sesión Spark
spark = SparkSession.builder.appName("MallCustomersAnalysis").getOrCreate()

# Leer el archivo CSV usando la ruta copiada
file_path = "dbfs:/FileStore/shared_uploads/ismaelarista.28@gmail.com/Mall_Customers.csv"
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_path)

# Crear una vista temporal
df.createOrReplaceTempView("customers_view")

# Ver los primeros registros de la vista temporal
spark.sql("SELECT * FROM customers_view LIMIT 5").show()



+----------+------+---+------------------+----------------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
+----------+------+---+------------------+----------------------+



In [0]:
# Consulta 1: Ver los primeros 10 registros
spark.sql("SELECT * FROM customers_view LIMIT 10").show()

# Consulta 2: Contar la cantidad de clientes según su género
spark.sql("""
    SELECT Genre, COUNT(*) AS CustomerCount
    FROM customers_view
    GROUP BY Genre
""").show()

# Consulta 3: Calcular la edad promedio de los clientes
spark.sql("""
    SELECT AVG(Age) AS AverageAge
    FROM customers_view
""").show()

# Consulta 4: Filtrar clientes con un ingreso anual mayor a 70,000
spark.sql("""
    SELECT *
    FROM customers_view
    WHERE `Annual Income (k$)` > 70
""").show()


+----------+------+---+------------------+----------------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
|         6|Female| 22|                17|                    76|
|         7|Female| 35|                18|                     6|
|         8|Female| 23|                18|                    94|
|         9|  Male| 64|                19|                     3|
|        10|Female| 30|                19|                    72|
+----------+------+---+------------------+----------------------+

+------+-------------+
| Genre|CustomerCount|
+------+-------------+
|Femal

Paso 4 de la Práctica

In [0]:
%sql
-- Consulta 1: Ver los primeros 10 registros
SELECT * FROM customers_view LIMIT 10;

-- Consulta 2: Contar la cantidad de clientes según su género
SELECT Genre, COUNT(*) AS CustomerCount
FROM customers_view
GROUP BY Genre;

-- Consulta 3: Calcular la edad promedio de los clientes
SELECT AVG(Age) AS AverageAge
FROM customers_view;

-- Consulta 4: Filtrar clientes con un ingreso anual mayor a 70,000
SELECT *
FROM customers_view
WHERE `Annual Income (k$)` > 70;

--CREATE OR REPLACE TEMPORARY VIEW customers_temp AS
--SELECT * FROM customers_view;

--SELECT * FROM customers_temp LIMIT 5;



CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
127,Male,43,71,35
128,Male,40,71,95
129,Male,59,71,11
130,Male,38,71,75
131,Male,47,71,9
132,Male,39,71,75
133,Female,25,72,34
134,Female,31,72,71
135,Male,20,73,5
136,Female,29,73,88


Paso 5 de la Práctica

In [0]:
# Ejecutar consulta y guardar los resultados en DataFrame
results_df = spark.sql("""
    SELECT Genre, AVG(Age) AS AverageAge, AVG(`Annual Income (k$)`) AS AverageIncome
    FROM customers_view
    GROUP BY Genre
""")

# Ruta de almacenamiento para el archivo Parquet
parquet_output_path = "/FileStore/tables/customers_results.parquet"

# Guardar el DataFrame como un archivo Parquet
results_df.write.mode("overwrite").parquet(parquet_output_path)

# Listar los archivos guardados en la ruta
dbutils.fs.ls("/FileStore/tables/")
dbutils.fs.ls("/FileStore/tables/customers_results.parquet/")

# Leer el archivo Parquet guardado
loaded_df = spark.read.parquet("/FileStore/tables/customers_results.parquet")

# Mostrar los datos
loaded_df.show()



+------+------------------+-----------------+
| Genre|        AverageAge|    AverageIncome|
+------+------------------+-----------------+
|Female|38.098214285714285|            59.25|
|  Male| 39.80681818181818|62.22727272727273|
+------+------------------+-----------------+

